# 4.실패한 유저 리뷰 다시 뽑기 (로컬에서 실행 추천)

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

from tqdm import tqdm

import time
import pandas as pd

In [ ]:
chrome_options = Options()

chrome_options.add_argument('window-size= 1920,1080')
chrome_options.add_argument('--kiosk')

In [ ]:
'''
돌리기 전에 다시 확인 꼭 하기!!
'''

area = 'Gangnam'  # 구 설정
version = 0  # 파일 이름 안 겹치게, 각자 실행할 때마다 1씩 더해주기
start = 0 # not_save 뒤에 붙는 파일 번호

In [ ]:
failed_list = []  # 실패한 번호들 담을 리스트
userlink = pd.DataFrame()
data = pd.read_csv(f'./area_csv/{area}/rest_concat.csv')
url_list = list(data['url'].values)
print(len(url_list))
print(url_list[:5])

In [ ]:
# 크롬 화면 최소화 하고 진행해야함.(ctrl + 휠로 조정)

for idx in tqdm(list(map(int, open(f'./user_csv/{area}/notsaved_{start}.txt', 'r').read().split()))):
    _url = url_list[idx]

    try:
        driver = webdriver.Chrome(executable_path='/opt/ml/input/project/crawings/chromedriver', chrome_options=chrome_options)
        action = ActionChains(driver)
        print(_url)
        URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"
        driver.get(URL)
        time.sleep(2.5)
        count = 0
        flag = False
        while True:
            try: action.move_to_element(driver.find_element(By.CLASS_NAME, "lfH3O")).click().perform()
            except: break
            print("\r",count, end="")
            count+= 1
            if count >= 500: flag = True; break
        print("click 1/2 complete")
        time.sleep(2.5)
        try:
            #action.move_to_element(driver.find_elements(By.CLASS_NAME, "YeINN")[-1]).perform()  #선택 리뷰 창 + 버튼 누르기
            driver.find_element(By.CLASS_NAME, 'I8cuq').click()
        except: print("NO 선택리뷰창")
        time.sleep(2.5)
        action = ActionChains(driver)
        while True:
            if flag:
                break  ## 첫 번째에서 이미 flag=True이면, 굳이 두 번째 while문 진행할 필요 없을듯
            try:
                action.move_to_element(driver.find_element(By.CLASS_NAME, "lfH3O")).click().perform()
            except:
                break
            print("\r",count, end="")
            count+= 1
            if count >= 500: flag = True; break
        print("click 2/2 complete")
        #if flag:
        #    failed_list.append(idx); print('failed\n')
            
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        user = soup.find_all(class_='YeINN')
        link_list = [i.a['href'] for i in user]
        user_list = [i.text for i in user]
        #time.sleep(5)
        print(_url, len(link_list), len(user_list))
        userlink2 = pd.DataFrame({'link' : link_list, 'user' : user_list}, dtype = str)
        userlink2['rest'] = _url
        userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)
        userlink.to_csv(f'./user_csv/{area}/user_fail_{version}.csv', index=False)  # ex) Gangnam_schini_0.csv
    except:
        print(_url)
        failed_list.append(idx); print('failed\n')
        continue

print("End~")

In [ ]:
print(failed_list)  # 실패한 번호들 확인
open(f'./user_csv/{area}/notsaved.txt', 'w').write('\n'.join(map(str, failed_list)))  # 실패한 번호들 다시 저장